In [6]:
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch
import torch.nn as nn
import pickle

In [7]:
with open('../data/processed/dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
device

'mps'

In [8]:
dataset.keys()

dict_keys(['X_train', 'y_train', 'X_val', 'y_val', 'X_test', 'y_test', 'X_train_store_category_le', 'X_val_store_category_le', 'X_test_store_category_le'])

In [37]:
X_train = dataset['X_train'].drop(columns=["store_primary_category"], axis=1)
y_train = dataset['y_train']
X_test = dataset['X_test'].drop(columns=["store_primary_category"], axis=1)
y_test = dataset['y_test']
X_val = dataset['X_val'].drop(columns=["store_primary_category"], axis=1)
y_val = dataset['y_val']
X_train_store_category_le=dataset['X_train_store_category_le']
X_val_store_category_le=dataset['X_val_store_category_le']
X_test_store_category_le=dataset['X_test_store_category_le']

In [38]:
X_train

,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_partners,total_busy_partners,total_outstanding_orders,created_at_day,created_at_week_of_month,...,order_protocol_4,order_protocol_5,order_protocol_6,order_protocol_7,market_id_1.0,market_id_2.0,market_id_3.0,market_id_4.0,market_id_5.0,market_id_6.0
0,-0.083488,-0.728377,0.197429,-1.052785,-0.562860,-0.604955,-0.582900,-0.518799,-1.024156,-1.235470,...,-0.328540,-0.553073,-0.056338,-0.0117,-0.501884,-0.669985,-0.363333,1.656367,-0.336232,-0.060702
1,-0.083488,0.529496,0.197429,0.165332,0.057817,1.668653,1.546290,1.287496,-0.485997,-0.504612,...,-0.328540,-0.553073,-0.056338,-0.0117,-0.501884,-0.669985,-0.363333,1.656367,-0.336232,-0.060702
2,-0.886190,-0.505914,-1.034719,1.373705,0.417630,1.697802,1.139239,1.572701,-0.916524,-1.235470,...,-0.328540,-0.553073,-0.056338,-0.0117,-0.501884,-0.669985,-0.363333,1.656367,-0.336232,-0.060702
3,-0.886190,-1.322159,-1.034719,-1.006009,-1.779028,1.668653,1.389732,1.553687,0.805585,0.957104,...,3.043768,-0.553073,-0.056338,-0.0117,-0.501884,1.492571,-0.363333,-0.603731,-0.336232,-0.060702
4,-0.083488,-0.304325,0.197429,-0.906611,0.424826,-1.042188,-1.021262,-0.937100,-0.270733,-0.504612,...,-0.328540,-0.553073,-0.056338,-0.0117,-0.501884,-0.669985,-0.363333,1.656367,-0.336232,-0.060702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124206,-0.083488,0.004925,0.197429,0.009413,-0.122089,1.085677,0.481695,1.211442,1.559007,1.687962,...,-0.328540,1.808080,-0.056338,-0.0117,-0.501884,-0.669985,-0.363333,1.656367,-0.336232,-0.060702
124207,-0.484839,-0.401549,-0.418645,0.009413,0.201742,-1.275378,-1.271755,-1.070195,-0.916524,-1.235470,...,-0.328540,-0.553073,-0.056338,-0.0117,-0.501884,-0.669985,2.752298,-0.603731,-0.336232,-0.060702
124208,0.719214,0.754705,1.429577,-0.575283,-0.122089,0.735891,0.794811,0.926237,-0.485997,-0.504612,...,3.043768,-0.553073,-0.056338,-0.0117,-0.501884,-0.669985,-0.363333,1.656367,-0.336232,-0.060702
124209,0.317863,-0.302677,0.197429,-0.857886,-0.607837,1.377165,1.201862,1.135387,0.590321,0.226246,...,-0.328540,1.808080,-0.056338,-0.0117,-0.501884,-0.669985,-0.363333,1.656367,-0.336232,-0.060702


In [39]:
class CustomDataset(Dataset):
    def __init__(self, X, y, category):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.long).unsqueeze(1)
        self.category = torch.tensor(category.values, dtype=torch.long).unsqueeze(1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx], self.category[idx]
    
train_dataset = CustomDataset(X_train, y_train, X_train_store_category_le)
val_dataset = CustomDataset(X_val, y_val, X_val_store_category_le)
test_dataset = CustomDataset(X_test, y_test, X_test_store_category_le)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [40]:
for X_batch, y_batch, category_batch in val_loader:
    print(f'X_batch shape: {X_batch.shape}, y_batch shape: {y_batch.shape}, category_batch shape: {category_batch.shape}')
    break

X_batch shape: torch.Size([64, 29]), y_batch shape: torch.Size([64, 1]), category_batch shape: torch.Size([64, 1])


In [41]:
X_train.shape[1]

29

In [42]:
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 56)
        self.fc2 = nn.Linear(56, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = SimpleNN(X_train.shape[1]).to(device)
print(model)

SimpleNN(
  (fc1): Linear(in_features=29, out_features=56, bias=True)
  (fc2): Linear(in_features=56, out_features=1, bias=True)
)


In [43]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
def train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for X_batch, y_batch, category_batch in train_loader:
            X_batch, y_batch, category_batch = X_batch.to(device), y_batch.to(device), category_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch.float())
            loss.backward()
            optimizer.step()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_val_batch, y_val_batch, category_val_batch in val_loader:
                X_val_batch, y_val_batch, category_val_batch = X_val_batch.to(device), y_val_batch.to(device), category_val_batch.to(device)
                y_val_pred = model(X_val_batch)
                val_loss += loss_fn(y_val_pred, y_val_batch.float()).item()
        val_loss /= len(val_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}')
    
    return model

num_epochs = 10
model = train_model(model, train_loader, val_loader, loss_fn, optimizer, num_epochs=num_epochs)

Epoch 1/10, Validation Loss: 272.7763
Epoch 2/10, Validation Loss: 256.3637
Epoch 3/10, Validation Loss: 251.5041
Epoch 4/10, Validation Loss: 248.9433
Epoch 5/10, Validation Loss: 247.3906
Epoch 6/10, Validation Loss: 247.0163
Epoch 7/10, Validation Loss: 244.4035
Epoch 8/10, Validation Loss: 243.7940
Epoch 9/10, Validation Loss: 242.4462
Epoch 10/10, Validation Loss: 242.1673
